The code replicates the Vision Tranfomer Paper named:

**An Image is Worth 16x16 Words: Transformers for Image Recognition at Scale**

https://arxiv.org/abs/2010.11929

In [5]:
# Import key libraries
import torch
import torch.nn as nn
import torch.optim
import torch.nn.functional as F
import torch.random
from torch.utils.data import DataLoader, TensorDataset

import torchvision
import torchvision.transforms as T
from torchvision.transforms import v2
import torchvision.models as models
import torchvision.datasets as datasets

from PIL import Image

import shutil
import subprocess
from pathlib import Path
import os
import sys
from google.colab import drive
import requests
import zipfile
from timeit import default_timer as timer
import time
import random

from tqdm.auto import tqdm

from typing import List, Tuple, Dict, Optional

import numpy as np

import matplotlib.pyplot as plt

try:
  from torchinfo import summary
except:
  subprocess.run(["pip", "install", "torchinfo"],check=True)
  from torchinfo import summary


URL = "https://github.com/anirguha/Python-2/raw/refs/heads/master/Helper%20Functions/copy_helper_functions.py"
code = requests.get(URL, timeout=30).text
exec(compile(code, URL, "exec"), {"__name__": "__main__"})

if os.path.exists("/content/Helper Functions"):
  for dirpath, dirnames, filenames in os.walk("/content/Helper Functions", topdown=True):
    sys.path.append(dirpath)
else:
  print("\u274c Error in loading Helper functions ")

import data_setup, engine, model_builder, utils, predict

from helpers import download_data, set_seeds, plot_loss_curves


device = "cuda" if torch.cuda.is_available() else "cpu"


Helper Functions already exists... deleting.
Copying folder to working directory ...
Cleaning up cloned repository ...
✅ Copy complete!


In [6]:
image_url = "https://github.com/anirguha/Python-2/raw/refs/heads/master/Pytorch%20for%20Deep%20Learning%20Bootcamp/data/pizza_steak_sushi.zip"

image_path = download_data(image_url, "pizza_steak_sushi")

[INFO] Did not find data/pizza_steak_sushi directory, creating one...
[INFO] Downloading pizza_steak_sushi.zip from https://github.com/anirguha/Python-2/raw/refs/heads/master/Pytorch%20for%20Deep%20Learning%20Bootcamp/data/pizza_steak_sushi.zip...
[INFO] Unzipping pizza_steak_sushi.zip data...


In [12]:
train_dir = image_path / "train"
test_dir = image_path / "test"

(PosixPath('data/pizza_steak_sushi/train'),
 PosixPath('data/pizza_steak_sushi/test'))

# Create DataLoaders

In [26]:
# Transform the data
IMAGE_SIZE = (224,224)
transforms = v2.Compose([
    v2.Resize(IMAGE_SIZE),
    v2.ToImage(),
    v2.ToDtype(torch.float32, scale=True)
])

In [39]:
import importlib
importlib.reload(data_setup)

<module 'data_setup' from '/content/Helper Functions/Important_PyTorch_ Modules/data_setup.py'>

In [40]:
# Create dataloaders
BATCH_SIZE = 32
NUM_WORKERS = os.cpu_count()

train_dataloader, test_dataloader, class_names = data_setup.create_dataloaders(
    train_dir=train_dir,
    test_dir=test_dir,
    transform=transforms,
    batch_size=BATCH_SIZE,
    num_workers=NUM_WORKERS,
    pin_memory=True,
    device=device
)

In [ ]:
os.cp